In [1]:
# Deep Convolutional GANs
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
# Setting some hyperparameters
batchSize = 64 # size of the batch.
imageSize = 64 # size of the generated images (64x64).

In [3]:
# Transformations
transform = transforms.Compose([transforms.Scale(imageSize), 
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) 

D:\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:208: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [4]:
# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform) 
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) 

Files already downloaded and verified


In [5]:
# weights_init function inputs a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [6]:
# Generator
class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input)
        return output

In [7]:
# Creating the generator
netG = G()
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [8]:
# Discriminator
class D(nn.Module):

    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1)

In [9]:
# Creating the discriminator
netD = D()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [10]:
# Training the DCGANs

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))


In [11]:
for epoch in range(25):

    for i, data in enumerate(dataloader, 0):
        
        # Updating the weights of the neural network of the discriminator
        netD.zero_grad()
        
        # Training the discriminator with a real image of the dataset
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output, target)
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())
        errD_fake = criterion(output, target)
        
        # Backpropagating the total error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        # Updating the weights of the neural network of the generator
        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        # Losses and saving the real images and the generated images of the minibatch every 100 steps

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.data, errG.data))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)

[0/25][0/782] Loss_D: 1.3600 Loss_G: 5.2260
[0/25][1/782] Loss_D: 1.3139 Loss_G: 6.4804
[0/25][2/782] Loss_D: 0.8382 Loss_G: 6.4210
[0/25][3/782] Loss_D: 0.9712 Loss_G: 5.7027
[0/25][4/782] Loss_D: 1.0001 Loss_G: 7.1403
[0/25][5/782] Loss_D: 0.7019 Loss_G: 6.6514
[0/25][6/782] Loss_D: 0.6740 Loss_G: 7.9681
[0/25][7/782] Loss_D: 0.7526 Loss_G: 7.2783
[0/25][8/782] Loss_D: 1.2442 Loss_G: 9.0363
[0/25][9/782] Loss_D: 0.6163 Loss_G: 8.1939
[0/25][10/782] Loss_D: 0.6773 Loss_G: 9.0408
[0/25][11/782] Loss_D: 0.5018 Loss_G: 9.2133
[0/25][12/782] Loss_D: 0.6359 Loss_G: 9.9307
[0/25][13/782] Loss_D: 0.3574 Loss_G: 10.1466
[0/25][14/782] Loss_D: 0.4328 Loss_G: 9.6973
[0/25][15/782] Loss_D: 0.5438 Loss_G: 12.4498
[0/25][16/782] Loss_D: 0.3627 Loss_G: 8.6236
[0/25][17/782] Loss_D: 1.0239 Loss_G: 16.4972
[0/25][18/782] Loss_D: 0.3013 Loss_G: 15.0254
[0/25][19/782] Loss_D: 0.2849 Loss_G: 7.6603
[0/25][20/782] Loss_D: 2.6783 Loss_G: 18.6489
[0/25][21/782] Loss_D: 0.2512 Loss_G: 18.6064
[0/25][22/782]

[0/25][355/782] Loss_D: 0.9929 Loss_G: 3.1899
[0/25][356/782] Loss_D: 0.7021 Loss_G: 3.4127
[0/25][357/782] Loss_D: 0.7313 Loss_G: 2.7069
[0/25][358/782] Loss_D: 0.8743 Loss_G: 4.4595
[0/25][359/782] Loss_D: 0.4851 Loss_G: 3.1965
[0/25][360/782] Loss_D: 0.5253 Loss_G: 3.6456
[0/25][361/782] Loss_D: 0.9904 Loss_G: 2.7607
[0/25][362/782] Loss_D: 0.8769 Loss_G: 6.6304
[0/25][363/782] Loss_D: 1.4744 Loss_G: 1.1259
[0/25][364/782] Loss_D: 1.8068 Loss_G: 8.1190
[0/25][365/782] Loss_D: 2.5321 Loss_G: 2.7468
[0/25][366/782] Loss_D: 1.2523 Loss_G: 3.0429
[0/25][367/782] Loss_D: 1.2170 Loss_G: 4.4519
[0/25][368/782] Loss_D: 1.4866 Loss_G: 2.2019
[0/25][369/782] Loss_D: 1.0278 Loss_G: 4.6334
[0/25][370/782] Loss_D: 1.0842 Loss_G: 2.7072
[0/25][371/782] Loss_D: 0.6391 Loss_G: 3.7253
[0/25][372/782] Loss_D: 0.4472 Loss_G: 4.0221
[0/25][373/782] Loss_D: 0.4379 Loss_G: 3.0442
[0/25][374/782] Loss_D: 0.6426 Loss_G: 3.3109
[0/25][375/782] Loss_D: 0.6792 Loss_G: 3.7888
[0/25][376/782] Loss_D: 0.6513 Los

[0/25][711/782] Loss_D: 0.4493 Loss_G: 3.0589
[0/25][712/782] Loss_D: 0.4591 Loss_G: 2.5617
[0/25][713/782] Loss_D: 0.4938 Loss_G: 3.3058
[0/25][714/782] Loss_D: 0.4598 Loss_G: 3.1156
[0/25][715/782] Loss_D: 0.4482 Loss_G: 2.5503
[0/25][716/782] Loss_D: 0.4964 Loss_G: 4.1367
[0/25][717/782] Loss_D: 0.3185 Loss_G: 3.9142
[0/25][718/782] Loss_D: 0.3234 Loss_G: 2.4931
[0/25][719/782] Loss_D: 0.6221 Loss_G: 4.6962
[0/25][720/782] Loss_D: 0.2742 Loss_G: 4.0318
[0/25][721/782] Loss_D: 0.2314 Loss_G: 2.5095
[0/25][722/782] Loss_D: 0.4237 Loss_G: 4.8209
[0/25][723/782] Loss_D: 0.4045 Loss_G: 3.2849
[0/25][724/782] Loss_D: 0.4286 Loss_G: 3.7176
[0/25][725/782] Loss_D: 0.4306 Loss_G: 3.1854
[0/25][726/782] Loss_D: 0.2988 Loss_G: 4.2946
[0/25][727/782] Loss_D: 0.3823 Loss_G: 3.4790
[0/25][728/782] Loss_D: 0.4353 Loss_G: 2.2542
[0/25][729/782] Loss_D: 0.8060 Loss_G: 6.4042
[0/25][730/782] Loss_D: 1.6468 Loss_G: 0.7416
[0/25][731/782] Loss_D: 1.6194 Loss_G: 4.7941
[0/25][732/782] Loss_D: 0.3446 Los

[1/25][287/782] Loss_D: 0.4290 Loss_G: 3.4589
[1/25][288/782] Loss_D: 0.6272 Loss_G: 3.4913
[1/25][289/782] Loss_D: 0.3640 Loss_G: 5.1954
[1/25][290/782] Loss_D: 0.5587 Loss_G: 2.8402
[1/25][291/782] Loss_D: 0.4521 Loss_G: 4.9608
[1/25][292/782] Loss_D: 0.2677 Loss_G: 5.1406
[1/25][293/782] Loss_D: 0.5946 Loss_G: 2.5849
[1/25][294/782] Loss_D: 0.9849 Loss_G: 5.7437
[1/25][295/782] Loss_D: 0.4319 Loss_G: 4.5126
[1/25][296/782] Loss_D: 0.5762 Loss_G: 2.4073
[1/25][297/782] Loss_D: 0.7329 Loss_G: 5.4311
[1/25][298/782] Loss_D: 0.6668 Loss_G: 3.5340
[1/25][299/782] Loss_D: 0.5660 Loss_G: 3.2530
[1/25][300/782] Loss_D: 0.4322 Loss_G: 4.6499
[1/25][301/782] Loss_D: 0.5188 Loss_G: 3.3558
[1/25][302/782] Loss_D: 0.4030 Loss_G: 4.9144
[1/25][303/782] Loss_D: 0.3944 Loss_G: 3.7439
[1/25][304/782] Loss_D: 0.2503 Loss_G: 3.2619
[1/25][305/782] Loss_D: 0.4727 Loss_G: 4.4091
[1/25][306/782] Loss_D: 0.2644 Loss_G: 4.0323
[1/25][307/782] Loss_D: 0.3264 Loss_G: 4.2027
[1/25][308/782] Loss_D: 0.3797 Los

[1/25][643/782] Loss_D: 0.4686 Loss_G: 2.9556
[1/25][644/782] Loss_D: 0.6558 Loss_G: 4.7443
[1/25][645/782] Loss_D: 0.8970 Loss_G: 1.5328
[1/25][646/782] Loss_D: 0.9230 Loss_G: 5.1338
[1/25][647/782] Loss_D: 0.4540 Loss_G: 4.0101
[1/25][648/782] Loss_D: 0.4722 Loss_G: 2.6693
[1/25][649/782] Loss_D: 0.4213 Loss_G: 4.1591
[1/25][650/782] Loss_D: 0.3246 Loss_G: 4.1748
[1/25][651/782] Loss_D: 0.7028 Loss_G: 2.0969
[1/25][652/782] Loss_D: 0.8431 Loss_G: 4.6088
[1/25][653/782] Loss_D: 0.3991 Loss_G: 4.2011
[1/25][654/782] Loss_D: 0.6675 Loss_G: 1.9402
[1/25][655/782] Loss_D: 0.7847 Loss_G: 4.7162
[1/25][656/782] Loss_D: 1.1075 Loss_G: 1.5252
[1/25][657/782] Loss_D: 0.8653 Loss_G: 3.3703
[1/25][658/782] Loss_D: 0.5605 Loss_G: 3.3600
[1/25][659/782] Loss_D: 0.6076 Loss_G: 2.5276
[1/25][660/782] Loss_D: 0.9604 Loss_G: 5.6088
[1/25][661/782] Loss_D: 0.8779 Loss_G: 3.2882
[1/25][662/782] Loss_D: 0.3791 Loss_G: 3.0380
[1/25][663/782] Loss_D: 0.9321 Loss_G: 5.6050
[1/25][664/782] Loss_D: 0.9397 Los

...

[22/25][761/782] Loss_D: 0.3227 Loss_G: 3.5608
[22/25][762/782] Loss_D: 0.2965 Loss_G: 2.6043
[22/25][763/782] Loss_D: 0.5174 Loss_G: 4.5055
[22/25][764/782] Loss_D: 0.4579 Loss_G: 2.5890
[22/25][765/782] Loss_D: 0.3949 Loss_G: 3.1779
[22/25][766/782] Loss_D: 0.3878 Loss_G: 3.9137
[22/25][767/782] Loss_D: 0.4032 Loss_G: 3.1621
[22/25][768/782] Loss_D: 0.4085 Loss_G: 3.4659
[22/25][769/782] Loss_D: 0.7098 Loss_G: 1.4558
[22/25][770/782] Loss_D: 0.4783 Loss_G: 4.4786
[22/25][771/782] Loss_D: 0.6467 Loss_G: 2.6513
[22/25][772/782] Loss_D: 0.3751 Loss_G: 2.6448
[22/25][773/782] Loss_D: 0.3207 Loss_G: 2.8114
[22/25][774/782] Loss_D: 0.3318 Loss_G: 3.8729
[22/25][775/782] Loss_D: 0.1829 Loss_G: 4.1869
[22/25][776/782] Loss_D: 0.5930 Loss_G: 1.4222
[22/25][777/782] Loss_D: 0.9965 Loss_G: 5.3812
[22/25][778/782] Loss_D: 1.0293 Loss_G: 1.6123
[22/25][779/782] Loss_D: 0.7145 Loss_G: 4.2489
[22/25][780/782] Loss_D: 0.4107 Loss_G: 3.2096
[22/25][781/782] Loss_D: 0.3327 Loss_G: 5.0010
[23/25][0/782

[23/25][329/782] Loss_D: 0.4799 Loss_G: 7.3819
[23/25][330/782] Loss_D: 3.5073 Loss_G: 0.0778
[23/25][331/782] Loss_D: 3.6173 Loss_G: 6.7989
[23/25][332/782] Loss_D: 1.4818 Loss_G: 0.7203
[23/25][333/782] Loss_D: 1.2504 Loss_G: 4.9169
[23/25][334/782] Loss_D: 0.7396 Loss_G: 2.7305
[23/25][335/782] Loss_D: 0.6176 Loss_G: 1.7601
[23/25][336/782] Loss_D: 0.6748 Loss_G: 3.0515
[23/25][337/782] Loss_D: 0.3065 Loss_G: 3.6588
[23/25][338/782] Loss_D: 0.7462 Loss_G: 1.5272
[23/25][339/782] Loss_D: 0.6313 Loss_G: 3.5220
[23/25][340/782] Loss_D: 0.4571 Loss_G: 3.2382
[23/25][341/782] Loss_D: 0.5345 Loss_G: 2.5208
[23/25][342/782] Loss_D: 0.4401 Loss_G: 2.5101
[23/25][343/782] Loss_D: 0.5153 Loss_G: 5.2631
[23/25][344/782] Loss_D: 0.4790 Loss_G: 2.6167
[23/25][345/782] Loss_D: 0.2772 Loss_G: 3.4542
[23/25][346/782] Loss_D: 0.2269 Loss_G: 4.6331
[23/25][347/782] Loss_D: 0.4558 Loss_G: 1.7350
[23/25][348/782] Loss_D: 0.7842 Loss_G: 8.1619
[23/25][349/782] Loss_D: 2.3264 Loss_G: 1.1351
[23/25][350/7

[23/25][677/782] Loss_D: 0.6694 Loss_G: 4.5506
[23/25][678/782] Loss_D: 0.7423 Loss_G: 1.5841
[23/25][679/782] Loss_D: 0.7039 Loss_G: 3.6927
[23/25][680/782] Loss_D: 0.3999 Loss_G: 3.2317
[23/25][681/782] Loss_D: 0.5234 Loss_G: 1.5979
[23/25][682/782] Loss_D: 0.8091 Loss_G: 4.9492
[23/25][683/782] Loss_D: 0.8846 Loss_G: 2.0802
[23/25][684/782] Loss_D: 0.6309 Loss_G: 3.0465
[23/25][685/782] Loss_D: 0.5510 Loss_G: 3.4958
[23/25][686/782] Loss_D: 0.4104 Loss_G: 2.4300
[23/25][687/782] Loss_D: 0.4601 Loss_G: 3.0194
[23/25][688/782] Loss_D: 0.4318 Loss_G: 3.3601
[23/25][689/782] Loss_D: 0.2242 Loss_G: 3.7462
[23/25][690/782] Loss_D: 0.8337 Loss_G: 1.0822
[23/25][691/782] Loss_D: 1.2570 Loss_G: 4.9990
[23/25][692/782] Loss_D: 1.0225 Loss_G: 1.5043
[23/25][693/782] Loss_D: 0.5692 Loss_G: 3.2195
[23/25][694/782] Loss_D: 0.8691 Loss_G: 0.9968
[23/25][695/782] Loss_D: 0.9934 Loss_G: 4.1470
[23/25][696/782] Loss_D: 0.2857 Loss_G: 4.1330
[23/25][697/782] Loss_D: 0.5873 Loss_G: 1.7320
[23/25][698/7

[24/25][245/782] Loss_D: 0.6032 Loss_G: 2.6803
[24/25][246/782] Loss_D: 0.7506 Loss_G: 6.5073
[24/25][247/782] Loss_D: 0.9107 Loss_G: 2.3611
[24/25][248/782] Loss_D: 0.4996 Loss_G: 4.4592
[24/25][249/782] Loss_D: 0.5000 Loss_G: 2.2118
[24/25][250/782] Loss_D: 0.4602 Loss_G: 5.5969
[24/25][251/782] Loss_D: 0.3258 Loss_G: 3.5977
[24/25][252/782] Loss_D: 0.2370 Loss_G: 4.3081
[24/25][253/782] Loss_D: 0.1142 Loss_G: 4.5346
[24/25][254/782] Loss_D: 0.1933 Loss_G: 3.1375
[24/25][255/782] Loss_D: 0.1072 Loss_G: 4.3310
[24/25][256/782] Loss_D: 0.0516 Loss_G: 4.8744
[24/25][257/782] Loss_D: 0.1720 Loss_G: 4.7968
[24/25][258/782] Loss_D: 0.0706 Loss_G: 5.0009
[24/25][259/782] Loss_D: 0.0381 Loss_G: 4.7796
[24/25][260/782] Loss_D: 0.0705 Loss_G: 4.3192
[24/25][261/782] Loss_D: 0.0525 Loss_G: 4.4820
[24/25][262/782] Loss_D: 0.0567 Loss_G: 5.0920
[24/25][263/782] Loss_D: 0.0404 Loss_G: 5.2824
[24/25][264/782] Loss_D: 0.1091 Loss_G: 5.1132
[24/25][265/782] Loss_D: 0.1262 Loss_G: 4.7175
[24/25][266/7

[24/25][593/782] Loss_D: 0.3208 Loss_G: 4.7557
[24/25][594/782] Loss_D: 0.2485 Loss_G: 3.9199
[24/25][595/782] Loss_D: 0.2248 Loss_G: 2.3567
[24/25][596/782] Loss_D: 0.2344 Loss_G: 4.4144
[24/25][597/782] Loss_D: 0.1830 Loss_G: 3.8870
[24/25][598/782] Loss_D: 0.0552 Loss_G: 4.4960
[24/25][599/782] Loss_D: 0.0783 Loss_G: 4.1737
[24/25][600/782] Loss_D: 0.1172 Loss_G: 3.4086
[24/25][601/782] Loss_D: 0.1907 Loss_G: 5.3320
[24/25][602/782] Loss_D: 0.2312 Loss_G: 3.1321
[24/25][603/782] Loss_D: 0.2311 Loss_G: 6.2903
[24/25][604/782] Loss_D: 0.6097 Loss_G: 1.8841
[24/25][605/782] Loss_D: 0.6476 Loss_G: 7.8785
[24/25][606/782] Loss_D: 2.3105 Loss_G: 0.2466
[24/25][607/782] Loss_D: 2.1768 Loss_G: 8.5068
[24/25][608/782] Loss_D: 2.2498 Loss_G: 1.0561
[24/25][609/782] Loss_D: 0.8577 Loss_G: 2.7233
[24/25][610/782] Loss_D: 0.5338 Loss_G: 3.1425
[24/25][611/782] Loss_D: 0.8550 Loss_G: 5.5852
[24/25][612/782] Loss_D: 2.3484 Loss_G: 0.4566
[24/25][613/782] Loss_D: 1.7597 Loss_G: 3.2193
[24/25][614/7

<img src="results/fake_samples_epoch_024.png">